In [1]:
from langchain.document_loaders import PyPDFLoader

document_loader = PyPDFLoader("C:/coloque aqui o seu caminho/multicreterioAHP-TOPSIS_v2.pdf")
documents = document_loader.load()
# Dividir os documentos em trechos menores

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

In [3]:
chunks = text_splitter.split_documents(documents)

In [4]:
# Criar uma Vector Store com embeddings da OpenAI
from langchain_openai import OpenAIEmbeddings

import os

os.environ["OPENAI_API_KEY"] = "coloque aqui a sua chave api"

In [ ]:

from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [6]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma/'
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=persist_directory
)


In [7]:
print(vectordb._collection.count())

56


In [ ]:
#conectar com o LLM

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature= 0.0, openai_api_key="coloque aqui sua chave api")


In [13]:
retriever = vectordb.as_retriever(search_kwargs={"k": 8})

In [46]:
#construimos o template em ingles porque o doc pdf está em ingles
prompt_template = """
You are a doctoral student on the decision sciences management course at the University of Coimbra and are studying multi-criteria methods for decision-making.

Context: You have access to a work that discusses the Analytic Hierarchy Process and Technique for Order Preference by Similarity to Ideal Solution methods.
Use the information from the document and your training data to provide comprehensive answers within the scope of the study.

User question: {question}

Relevant information from documents:
{context}

Instructions:
1. Provide a detailed and complete answer to the user's question.
2. Use all relevant information from the provided context and your knowledge.
3. Ensure your response is well-structured and easy to understand.
4. If the information is insufficient, clearly state what's missing and provide the best possible answer with available data.
5. Include examples or explanations of key concepts when appropriate.

Answer:
"""


In [14]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key="coloque aqui sua chave api"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

C:\Users\ana.fjanuario\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
# Function to make a query
def make_query(query):
    try:
        result = qa_chain({"query": query})
        print("Answer:", result['result'])
        
    except Exception as e:
        print(f"Erro durante a execução: {e}")

In [16]:
make_query("Explain LI5 how the methos AHP works?")

C:\Users\ana.fjanuario\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer:  AHP stands for Analytic Hierarchy Process, which is a method for multi-criteria decision-making. It involves breaking down a complex decision into a hierarchy of criteria and alternatives, and then performing pairwise comparisons between them. This allows decision-makers to evaluate the importance of each criteria and the performance of each alternative. The method is based on mathematical principles and aims to reduce subjectivity in decision-making. However, it can be time-consuming and may be susceptible to biases in the weight assignment process.


In [17]:
make_query("when we use the AHP method with TOPSIS method?")

Answer:  The AHP method is used in conjunction with the TOPSIS method, meaning they are used together in a combined approach. This combination is often used in multi-criteria decision-making problems to produce a full ranking of alternatives. 


In [18]:
make_query("Considering the application of the method in the article, was it used well?")

Answer:  Yes, the AHP and TOPSIS methods were used well in the article, as they were applied in a clear and transparent manner and their strengths and limitations were discussed. However, the use of additional tests, such as Kendall's Tau, could have enhanced the validity of the results.


In [19]:
make_query("o documento analisado tem formato de artigo cientifico?") #fazer a pergunta em portugues para avaliar o output

Answer:  Yes. The paper under analysis has the format and structure of a scientific article. It includes a summary, keywords, introduction, theoretical background, practical application, discussion, and conclusion sections, as well as references and figures to support the information presented.


In [20]:
make_query("o que o autor diz sobre o MAVT?") #agora o LLM respondeu em portugues, mas não gostei da resposta dele

Answer: 

O autor menciona que o MAVT (Multi-Attribute Value Theory) é uma abordagem quantitativa para tomada de decisão que proporciona um quadro para avaliar e classificar alternativas com base no seu desempenho em vários atributos relevantes. Ele destaca que o MAVT incorpora diretamente as preferências e percepções dos tomadores de decisão, promovendo decisões que se alinham com os valores e objetivos organizacionais. No entanto, ele ressalta que o MAVT também possui desvantagens, como a sensibilidade aos pesos atribuídos aos critérios e a influência da escolha de alternativas de referência nos resultados de classificação. 


In [21]:
make_query("o autor explica o que é uma função de valor no texto?") #repondeu em ingles, entretanto respondeu bem

Answer:  Yes, the author explains what a value function is in the text. They define it as a mapping of performance levels to values between 0 and 1 and provide an example of a value function for cost. They also discuss the process of constructing value functions and the importance of understanding attributes and decision-makers' preferences in this process.


In [22]:
make_query("Suppose you are the author of this text, what would you include in this article to make it more complete?")

Answer: 
As the author of this text, I would include a section discussing the potential future applications and advancements of the AHP-TOPSIS methodology in credit scoring for SMEs. This could include discussing the potential for incorporating machine learning and artificial intelligence techniques, as well as the potential for expanding the methodology to other industries or types of decision-making processes. Additionally, I would include a discussion on the limitations and challenges of implementing the AHP-TOPSIS methodology in practice, and potential strategies for overcoming these challenges. Finally, I would include a section on the implications and recommendations for decision-makers and stakeholders, highlighting the key takeaways and actionable insights from the study.
